### Градиентный бустинг

В случае бустинга композиция алгоритмов строится последовательно. Каждый следующий базовый алгоритм акцентируется на тех объектах, на которых обученная ранее композиция допускала ошибку.

На данный момент одной из самых широко распространенных реализаций бустинга является библиотека [XGBoost](https://github.com/dmlc/xgboost). В ней большое внимание уделяется регуляризации и скорости, нежели в других реализациях бустинга (например,  [GradientBoostingRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html),  [GradientBoostingClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) из sklearn). Кроме того, XGBoost позволяет оптимизировать различные функции потерь, а также более гибок, засчет большого числа параметров.

XGBoost строит композицию из $K$ базовых алгоритмов $b_k$:

$$ \hat{y}_i = \hat{y}_i^{K} = \sum_{k=1}^{K} b_k(x_i) = \hat{y}_i^{\left(K - 1\right)} + b_K(x_i), $$

минимизируя следующий функционал:

$$ Obj = \sum_{i=1}^N \mathcal{L}(y_i, \hat{y}_i ) + \sum_{k=1}^{K} \Omega(b_k),$$

где
 - $N$ — размер обучающей выборки;
 - $x_i, y_i, \hat{y}_i$ — i-ый объект, правильный ответ и предсказание модели для него;
 - $\hat{y}_i^{t}$ — предсказание композиции из $t$ уже обученных базовых алгоритмов для i-го объекта;
 - $\Omega$ — регуляризатор;
 - $\mathcal{L}(y_i, \hat{y}_i)$ — функция потерь.

Функционал, оптимизируемый на $t$-ой итерации:

$$ Obj^{(t)} = \sum_{i=1}^N \mathcal{L}\left(y_i, \hat{y}_i^{(t-1)} + b_t(x_i)\right) + \Omega(b_t).$$

В XGBoost реализовано несколько различных функций потерь, что позволяет решать задачи классификации (бинарной и мультиклассовой), регрессии и ранжирования. Вот некоторые из них:

- reg:linear — линейная регрессия
- reg:logistic — логистическая регрессия
- binary:logistic — логистическая регрессия
- multi:softmax — softmax функция потерь для многоклассовой классификации
- rank:pairwise — минимизация pairwise-функции потерь для задачи ранжирования

In [1]:
# линейная алгебра
import numpy as np

# чтение файлов и обработка данных
import pandas as pd
from pandas import Series, DataFrame

# препроцессинг, метрики моделей, валидация моделей, выбор признаков, выбор моделей,
# модели машинного обучения: классификация, регрессия, кластеризция, понижение размерности...
import sklearn

## Регрессия

Возьмём датасет хат в Бостоне и попробуем предсказать их стоимость

In [2]:
from sklearn.datasets import load_boston

boston = load_boston()

X1, y1 = DataFrame(boston.data), Series(boston.target)

X1.shape, y1.shape

((506, 13), (506,))

Всего у нас 13 критериев

In [3]:
X1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


Правильные ответы для каждой из квартир

In [4]:
y1.head()

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
dtype: float64

Разобъём выборку с фиксированным сидом на обучающую и проверочную в соотношении 80% к 20%

In [5]:
from sklearn.model_selection import train_test_split

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

X_train1.shape, X_test1.shape, y_train1.shape, y_test1.shape

((404, 13), (102, 13), (404,), (102,))

Обучим метод XGBRegressor из библиотеки xgboost:

In [6]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error as mse

xgb = XGBRegressor(n_estimators=100)

xgb.fit(X_train1, y_train1)

y_pred1 = Series(xgb.predict(X_test1))

print(f"result of XGBRegressor is {mse(y_pred1, y_test1)}")

[15:32:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
result of XGBRegressor is 7.2667278403836315


Обучим метод GradientBoostingRegressor из библиотеки sklearn:

In [7]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error as mse

gbr = GradientBoostingRegressor(n_estimators=100)

gbr.fit(X_train1, y_train1)

y_pred2 = Series(gbr.predict(X_test1))

print(f"result of GradientBoostingRegressor is {mse(y_test1, y_pred2)}")

result of GradientBoostingRegressor is 6.209149129942637


Как мы видим, в зависимости от выбора фреймворка результат немного отличается:

In [8]:
err_df = DataFrame({
    "xgboost": y_pred1.values,
    "xgboost-from-sklearn": y_pred2.values,
    "true": y_test1.values
})

err_df.head()

,xgboost,xgboost-from-sklearn,true
0,23.691797,23.449761,23.6
1,31.711241,31.461360,32.4
2,16.610657,17.373812,13.6
3,23.944933,23.974831,22.8
4,16.949919,17.681144,16.1


## Классификация

Разберём задачу классификации на примере известной задачи о ирисах Фишера

In [9]:
from sklearn.datasets import load_iris

iris = load_iris()

X2, y2 = DataFrame(iris.data), Series(iris.target)

X2.shape, y2.shape

((150, 4), (150,))

Всего у нас 4 критерия:

In [10]:
X2.head()

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


Правильные ответы для каждого изучаемого ириса:

In [11]:
y2.head()

0    0
1    0
2    0
3    0
4    0
dtype: int64

Аналогично разобъём выборку:

In [12]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

X_train2.shape, X_test2.shape, y_train2.shape, y_test2.shape

((120, 4), (30, 4), (120,), (30,))

Обучим сразу две модели из самостоятельного фреймворка xgboost и из sklearn:

In [13]:
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import accuracy_score

xgb = XGBClassifier(n_estimators=100)

xgb.fit(X_train2, y_train2)

y_pred1 = Series(xgb.predict(X_test2))

print(f"result of XGBClassifier is {accuracy_score(y_pred1, y_test2)}")

gbr = GradientBoostingClassifier(n_estimators=100)

gbr.fit(X_train2, y_train2)

y_pred2 = Series(gbr.predict(X_test2))

print(f"result of GradientBoostingClassifier is {accuracy_score(y_pred2, y_test2)}")

result of XGBClassifier is 1.0
result of GradientBoostingClassifier is 1.0


В целом они оба отлично справились с задачей и правильно классифицировали ирисы:

In [14]:
err_df = DataFrame({
    "xgboost": y_pred1.values,
    "xgboost sklearn": y_pred2.values,
    "true": y_test2.values
})

err_df.head()

,xgboost,xgboost sklearn,true
0,1,1,1
1,0,0,0
2,2,2,2
3,1,1,1
4,1,1,1


## Оптимизация параметров

В sklearn есть отличная штука, которая позволяет перебирать параметры и выбирать наилучший для конкретной задачи.
Каждый параметр отлично расписан в официальной документации, ссылки ниже:

- Задача регресии на основе xgboost из sklearn: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html
- Задача классификации на основе xgboost из sklearn: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

- Задача регрессии на основе самостоятельного фреймворка xgboost: https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor
- Задача классификации на основе самостоятельного фреймворка xgboost: https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier

In [15]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'n_estimators': [100, 300, 500], 
    'learning_rate': [0.1, 0.01, 0.001],
#     'min_samples_leaf': [1, 3, 5],
#     'criterion': ('friedman_mse', 'mae', 'mse'),
#     'max_depth': [10, 20, 30]
}

gbr_opt = GridSearchCV(GradientBoostingRegressor(), parameters)

gbr_opt.fit(X_train1, y_train1)

GridSearchCV(cv=None, error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_iter_no_change=None,
              

In [16]:
y_pred3 = Series(gbr_opt.predict(X_test1))

print(f"result of GradientBoostingRegressor is {mse(y_test1, y_pred3)}")

result of GradientBoostingRegressor is 6.109083050296294
